In [214]:
!pip install torch torchvision opencv-python matplotlib

In [215]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
import time
import matplotlib.pyplot as plt
import cv2

In [184]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [216]:
# Пример использования
images_dir = "/content/drive/MyDrive/3_SEMESTR/CV/CVproject/ICDAR2013/Challenge2_Training_Task12_Images"  # Папка с изображениями
annotations_dir = "/content/drive/MyDrive/3_SEMESTR/CV/CVproject/ICDAR2013/Challenge2_Training_Task1_GT"  # Папка с аннотациями


In [217]:
def parse_annotation_file(file_path):
    """
    Читает файл с аннотациями и возвращает список bounding boxes и меток.
    """
    annotations = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            x1, y1, x2, y2 = map(int, parts[:4])
            label = " ".join(parts[4:]).strip('"')  # Обрабатываем метку, которая может содержать пробелы
            annotations.append((x1, y1, x2, y2, label))
    return annotations


In [218]:
import os
import random

# Убедимся, что изображения и аннотации соответствуют друг другу
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
annotation_files = [f for f in os.listdir(annotations_dir) if f.endswith('.txt')]

# Проверяем, что для каждого изображения есть аннотация
valid_image_files = []
for img_file in image_files:
    annotation_file = f"gt_{os.path.splitext(img_file)[0]}.txt"
    if annotation_file in annotation_files:
        valid_image_files.append(img_file)
    else:
        print(f"Аннотация для изображения {img_file} не найдена. Пропускаем.")

# Сортируем изображения для воспроизводимости
valid_image_files.sort()

# Разделяем изображения на обучающую и валидационную выборки
random.seed(42)
random.shuffle(valid_image_files)
split_idx = int(len(valid_image_files) * 0.9)
train_images = valid_image_files[:split_idx]
val_images = valid_image_files[split_idx:]

# Создаем списки аннотаций для обучающей и валидационной выборок
train_annotations = []
val_annotations = []

for img_file in train_images:
    annotation_file = f"gt_{os.path.splitext(img_file)[0]}.txt"
    annotation_path = os.path.join(annotations_dir, annotation_file)
    bboxes = parse_annotation_file(annotation_path)
    for bbox in bboxes:
        train_annotations.append({
            "image_name": img_file,
            "bbox": bbox,
        })

for img_file in val_images:
    annotation_file = f"gt_{os.path.splitext(img_file)[0]}.txt"
    annotation_path = os.path.join(annotations_dir, annotation_file)
    bboxes = parse_annotation_file(annotation_path)
    for bbox in bboxes:
        val_annotations.append({
            "image_name": img_file,
            "bbox": bbox,
        })

# Проверяем количество изображений и аннотаций
print(f"Количество изображений в обучающей выборке: {len(train_images)}")
print(f"Количество аннотаций в обучающей выборке: {len(train_annotations)}")
print(f"Количество изображений в валидационной выборке: {len(val_images)}")
print(f"Количество аннотаций в валидационной выборке: {len(val_annotations)}")

# Выводим первые 5 элементов обучающей выборки
print("\nПервые 5 элементов обучающей выборки:")
for i, ann in enumerate(train_annotations[:5]):
    print(f"Элемент {i + 1}:")
    print(f"  Изображение: {ann['image_name']}")
    print(f"  Bounding box: {ann['bbox']}")

# Выводим первые 5 элементов валидационной выборки
print("\nПервые 5 элементов валидационной выборки:")
for i, ann in enumerate(val_annotations[:5]):
    print(f"Элемент {i + 1}:")
    print(f"  Изображение: {ann['image_name']}")
    print(f"  Bounding box: {ann['bbox']}")

Количество изображений в обучающей выборке: 206
Количество аннотаций в обучающей выборке: 744
Количество изображений в валидационной выборке: 23
Количество аннотаций в валидационной выборке: 105

Первые 5 элементов обучающей выборки:
Элемент 1:
  Изображение: 144.jpg
  Bounding box: (1883, 439, 2358, 648, '2.17')
Элемент 2:
  Изображение: 144.jpg
  Bounding box: (1517, 698, 1814, 767, 'Knowledge')
Элемент 3:
  Изображение: 144.jpg
  Bounding box: (1838, 716, 2185, 777, 'Management')
Элемент 4:
  Изображение: 144.jpg
  Bounding box: (2206, 733, 2324, 793, '(KM)')
Элемент 5:
  Изображение: 144.jpg
  Bounding box: (1667, 1362, 1776, 1434, 'Dr.')

Первые 5 элементов валидационной выборки:
Элемент 1:
  Изображение: 254.jpg
  Bounding box: (561, 105, 604, 133, '2N')
Элемент 2:
  Изображение: 254.jpg
  Bounding box: (28, 118, 277, 178, '4B.522')
Элемент 3:
  Изображение: 254.jpg
  Bounding box: (22, 308, 177, 348, 'Marisa')
Элемент 4:
  Изображение: 254.jpg
  Bounding box: (199, 308, 399, 348

In [219]:
# Функция для нормализации изображения
def normalize_image(image):
    """
    Нормализует изображение к диапазону [0, 1].
    """
    return image / 255.0

# Функция для обрезки bounding boxes
def clamp_boxes(boxes, image_width, image_height):
    """
    Обрезает bounding boxes, чтобы они не выходили за пределы изображения.
    """
    boxes[:, 0] = torch.clamp(boxes[:, 0], min=0, max=image_width)  # x1
    boxes[:, 1] = torch.clamp(boxes[:, 1], min=0, max=image_height)  # y1
    boxes[:, 2] = torch.clamp(boxes[:, 2], min=0, max=image_width)  # x2
    boxes[:, 3] = torch.clamp(boxes[:, 3], min=0, max=image_height)  # y2
    return boxes

def filter_invalid_boxes(boxes):
    """
    Удаляет bounding boxes с нулевой шириной или высотой.
    """
    widths = boxes[:, 2] - boxes[:, 0]
    heights = boxes[:, 3] - boxes[:, 1]
    valid_indices = (widths > 0) & (heights > 0)
    return boxes[valid_indices]

In [220]:
# Класс датасета
class ICDARDataset(Dataset):
    def __init__(self, root, annotations, resize_to=(800, 800)):
        """
        Args:
            root (str): Путь к папке с изображениями.
            annotations (list): Список аннотаций.
            resize_to (tuple): Размер, к которому будут изменены изображения и bounding boxes.
        """
        self.root = root
        self.annotations = annotations
        self.imgs = list(set([ann["image_name"] for ann in annotations]))  # Уникальные имена изображений
        self.resize_to = resize_to

    def __getitem__(self, idx):
        # Загрузка изображения
        img_name = self.imgs[idx]
        img_path = os.path.join(self.root, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Преобразуем в RGB
        img = normalize_image(img)  # Нормализация
        img_height, img_width = img.shape[:2]

        # Получаем аннотации для текущего изображения
        anns = [ann for ann in self.annotations if ann["image_name"] == img_name]
        boxes = []
        labels = []
        for ann in anns:
            x1, y1, x2, y2, label = ann["bbox"]
            boxes.append([x1, y1, x2, y2])
            labels.append(1)  # Все объекты относятся к классу "text"

        # Преобразуем bounding boxes в тензор
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # Обрезаем bounding boxes, чтобы они не выходили за пределы изображения
        boxes = clamp_boxes(boxes, img_width, img_height)

        # Удаляем некорректные bounding boxes
        boxes = filter_invalid_boxes(boxes)

        # Если bounding boxes отсутствуют, пропускаем это изображение
        if len(boxes) == 0:
            print(f"Нет валидных bounding boxes в изображении {img_name}. Пропускаем.")
            return None, None

        # Изменяем размер изображения и bounding boxes
        img, boxes = self.resize_image_and_boxes(img, boxes)

        # Преобразуем изображение в тензор (C, H, W)
        img = torch.from_numpy(img).permute(2, 0, 1).float()

        # Создаем target (целевые значения для модели)
        target = {
            "boxes": boxes,  # Bounding boxes в формате [x_min, y_min, x_max, y_max]
            "labels": torch.as_tensor(labels[:len(boxes)], dtype=torch.int64),  # Метки классов
            "image_id": torch.tensor([idx]),  # Уникальный идентификатор изображения
            "area": (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]),  # Площадь bounding boxes
            "iscrowd": torch.zeros((len(boxes),), dtype=torch.int64),  # Флаг "iscrowd" (0 для отдельных объектов)
        }

        return img, target

    def __len__(self):
        return len(self.imgs)

    def resize_image_and_boxes(self, image, boxes):
        """
        Изменяет размер изображения и масштабирует bounding boxes.
        """
        orig_height, orig_width = image.shape[:2]
        new_height, new_width = self.resize_to

        # Изменяем размер изображения
        image = cv2.resize(image, (new_width, new_height))

        # Масштабируем bounding boxes
        scale_x = new_width / orig_width
        scale_y = new_height / orig_height
        boxes[:, [0, 2]] *= scale_x
        boxes[:, [1, 3]] *= scale_y

        return image, boxes

In [221]:
def check_data_consistency(dataset):
    for i in range(len(dataset)):
        img, target = dataset[i]
        boxes = target['boxes']
        labels = target['labels']

        # Проверка координат bounding boxes
        if (boxes[:, 0] < 0).any() or (boxes[:, 1] < 0).any() or \
           (boxes[:, 2] > img.shape[2]).any() or (boxes[:, 3] > img.shape[1]).any():
            print(f"Некорректные bounding boxes в изображении {dataset.imgs[i]}: {boxes}")

        # Проверка меток
        if (labels < 0).any():
            print(f"Некорректные метки в изображении {dataset.imgs[i]}: {labels}")

In [222]:
train_dataset = ICDARDataset(root=images_dir, annotations=train_annotations, resize_to=(800, 800))
val_dataset = ICDARDataset(root=images_dir, annotations=val_annotations, resize_to=(800, 800))

# Проверяем корректность данных
check_data_consistency(train_dataset)
check_data_consistency(val_dataset)

def print_random_samples(dataset, name, num_samples=3):
    """
    Выводит информацию о случайных элементах датасета.
    """
    print(f"Случайные элементы датасета {name}:")
    indices = random.sample(range(len(dataset)), num_samples)
    for i, idx in enumerate(indices):
        img, target = dataset[idx]
        print(f"Элемент {i + 1}:")
        print(f"  Изображение: {dataset.imgs[idx]}")
        print(f"  Форма изображения: {img.shape}")
        print(f"  Количество bounding boxes: {len(target['boxes'])}")
        if len(target['boxes']) > 0:
            print(f"  Пример bounding box: {target['boxes'][0]}")
        else:
            print("  Пример bounding box: Нет bounding boxes")
        print(f"  Метки: {target['labels']}")
        print(f"  Площадь: {target['area']}")
        print(f"  iscrowd: {target['iscrowd']}")
        print(f"  ID изображения: {target['image_id']}")
        print("\n")

# Вывод информации о 3 случайных элементах каждого датасета
print_random_samples(train_dataset, "Train")
print_random_samples(val_dataset, "Validation")

Случайные элементы датасета Train:
Элемент 1:
  Изображение: 132.jpg
  Форма изображения: torch.Size([3, 800, 800])
  Количество bounding boxes: 10
  Пример bounding box: tensor([ 59.6708, 478.3951, 141.9753, 545.6790])
  Метки: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
  Площадь: tensor([ 5537.7754,  9263.8906,  5161.8145, 16881.5723, 11630.4717,  6757.9302,
         6014.7051, 23430.9395,  6469.4800,  9620.9893])
  iscrowd: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  ID изображения: tensor([54])


Элемент 2:
  Изображение: 234.jpg
  Форма изображения: torch.Size([3, 800, 800])
  Количество bounding boxes: 7
  Пример bounding box: tensor([116.8750, 264.1667, 180.0000, 309.1667])
  Метки: tensor([1, 1, 1, 1, 1, 1, 1])
  Площадь: tensor([ 2840.6250,  7612.4966,  6750.0000,  4542.1875,  7233.3320,  1884.3750,
        13952.0879])
  iscrowd: tensor([0, 0, 0, 0, 0, 0, 0])
  ID изображения: tensor([138])


Элемент 3:
  Изображение: 171.jpg
  Форма изображения: torch.Size([3, 800, 800])
  Количест

In [223]:
def collate_fn(batch):
    """
    Объединяет данные в батчи, пропуская изображения без валидных bounding boxes.
    """
    batch = [item for item in batch if item[0] is not None]  # Пропускаем None
    if len(batch) == 0:
        return None, None  # Если все изображения в батче пропущены
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return images, targets

In [224]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=collate_fn)

In [235]:
def calculate_iou(box1, box2):
    """
    Рассчитывает Intersection over Union (IoU) для двух bounding boxes.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    iou = inter_area / (box1_area + box2_area - inter_area + 1e-9)
    return iou

def calculate_metrics(pred_boxes, true_boxes, iou_threshold=0.5):
    """
    Рассчитывает Precision, Recall и F1-score для одного изображения.
    """
    if len(pred_boxes) == 0:
        return 0.0, 0.0, 0.0  # Если нет предсказаний, метрики равны 0

    if len(true_boxes) == 0:
        return 0.0, 0.0, 0.0  # Если нет истинных объектов, метрики равны 0

    # Матрица IoU между предсказанными и истинными bounding boxes
    iou_matrix = torch.zeros((len(pred_boxes), len(true_boxes)))
    for i, pred_box in enumerate(pred_boxes):
        for j, true_box in enumerate(true_boxes):
            iou_matrix[i, j] = calculate_iou(pred_box, true_box)

    # Жадное сопоставление: каждый истинный объект сопоставляется с одним предсказанным bounding box
    tp = 0
    matched_true_indices = set()  # Индексы истинных объектов, которые уже сопоставлены
    matched_pred_indices = set()  # Индексы предсказанных объектов, которые уже сопоставлены

    # Сначала сопоставляем предсказания с истинными объектами
    for j in range(len(true_boxes)):
        max_iou_idx = torch.argmax(iou_matrix[:, j]).item()
        if iou_matrix[max_iou_idx, j] >= iou_threshold and max_iou_idx not in matched_pred_indices:
            tp += 1
            matched_true_indices.add(j)
            matched_pred_indices.add(max_iou_idx)

    # Подсчет FP и FN
    fp = len(pred_boxes) - len(matched_pred_indices)  # False Positives
    fn = len(true_boxes) - len(matched_true_indices)  # False Negatives

    # Расчет метрик
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall + 1e-9)  # Добавляем 1e-9 для избежания деления на 0

    return precision, recall, f1

def calculate_batch_metrics(preds, targets, iou_threshold=0.5):
    """
    Рассчитывает средние Precision, Recall и F1-score для батча.
    """
    batch_precision = []
    batch_recall = []
    batch_f1 = []

    for pred, target in zip(preds, targets):
        pred_boxes = pred['boxes'].cpu().numpy()
        true_boxes = target['boxes'].cpu().numpy()

        precision, recall, f1 = calculate_metrics(pred_boxes, true_boxes, iou_threshold)
        batch_precision.append(precision)
        batch_recall.append(recall)
        batch_f1.append(f1)

    # Средние значения метрик по батчу
    avg_precision = sum(batch_precision) / len(batch_precision)
    avg_recall = sum(batch_recall) / len(batch_recall)
    avg_f1 = sum(batch_f1) / len(batch_f1)

    return avg_precision, avg_recall, avg_f1

In [230]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=2)

In [238]:
def train_model(model, train_dataloader, val_dataloader, num_epochs=10, lr=0.001):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
    train_losses = []
    metrics = {'precision': [], 'recall': [], 'f1': []}

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0
        start_time = time.time()

        for images, targets in train_dataloader:
            if images is None:  # Пропускаем пустые батчи
                continue

            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Вызов модели
            loss_dict = model(images, targets)

            # Суммируем потери из словаря
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            epoch_train_loss += losses.item()

        train_losses.append(epoch_train_loss / len(train_dataloader))

        # Валидация
        model.eval()
        epoch_precision = 0
        epoch_recall = 0
        epoch_f1 = 0

        with torch.no_grad():
            for images, targets in val_dataloader:
                if images is None:  # Пропускаем пустые батчи
                    continue

                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                # Вызов модели (в режиме оценки возвращает список предсказаний)
                preds = model(images)

                # Расчет метрик
                for pred, target in zip(preds, targets):
                    pred_boxes = pred['boxes'].cpu().numpy()
                    true_boxes = target['boxes'].cpu().numpy()
                    precision, recall, f1 = calculate_metrics(pred_boxes, true_boxes)
                    epoch_precision += precision
                    epoch_recall += recall
                    epoch_f1 += f1

        # Средние значения метрик
        metrics['precision'].append(epoch_precision / len(val_dataloader))
        metrics['recall'].append(epoch_recall / len(val_dataloader))
        metrics['f1'].append(epoch_f1 / len(val_dataloader))

        # Вывод результатов
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {train_losses[-1]:.4f}")
        print(f"Precision: {metrics['precision'][-1]:.4f}, Recall: {metrics['recall'][-1]:.4f}, F1: {metrics['f1'][-1]:.4f}")
        print(f"Time: {time.time() - start_time:.2f}s")

    # Построение графика loss
    plt.plot(train_losses, label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Сохранение модели
    torch.save(model.state_dict(), "faster_rcnn_text_detection.pth")
    print("Модель сохранена как 'faster_rcnn_text_detection.pth'")

    return model, train_losses, metrics

In [1]:
train_model(model, train_dataloader, val_dataloader, num_epochs=50)

NameError: name 'train_model' is not defined